[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Rinovative/alaska2-steganalysis/blob/main/ANN_Projekt_Rino_Albertin_Steganalyse.ipynb)  
_Interaktives Jupyter Notebook direkt im Browser öffnen (via Colab)_

In [ ]:
try:
    import google.colab
    in_colab = True
except ImportError:
    in_colab = False

if in_colab:
    # Nur in Colab ausführen
    !git clone https://github.com/Rinovative/alaska2-steganalysis.git
    import os
    os.chdir('alaska2-steganalysis')
    !pip install -q -r requirements_colab.txt

In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import ipywidgets as widgets
from pathlib import Path
from src import eda, util

## Datensatzinformation 
##### ***Work in Progress: Für die finale Version wird der Ersatzdatensatz noch auf dem Cluster mit erweitertem Suchraum ermittelt. Aktuell basiert der veröffentlichte Datensatz auf einer ersten CLIP-Embedding-Auswahl ähnlich zu ALASKA2, wobei der verwendete Suchraum noch relativ begrenzt ist. In der aktuellen Version sind viele der ausgewählten Bilder Pflanzenbilder, was auf die begrenzte Auswahl im Suchraum zurückzuführen ist.***

Dieses Projekt wurde primär auf dem **ALASKA2-Datensatz** entwickelt. Da ALASKA2 aus Lizenzgründen nicht öffentlich weitergegeben werden darf, kann er über die offizielle [Kaggle-Seite](https://www.kaggle.com/competitions/alaska2-image-steganalysis) selbstständig bezogen und im Verzeichnis `data/raw/alaska2-image-steganalysis/` entpackt werden.

Für Demonstrationszwecke wird ein **synthetischer Ersatzdatensatz** auf Basis von **PD12M** erstellt. Dieser ist öffentlich unter [Rinovative/pd12m_dct_based_synthetic_stegano](https://huggingface.co/datasets/Rinovative/pd12m_dct_based_synthetic_stegano) verfügbar und wird automatisch heruntergeladen.

Falls ein anderer Ausschnitt von PD12M gewünscht ist, kann die Funktion `build_pd12m_like_alaska2` verwendet werden. Dabei werden mithilfe von CLIP-Embeddings die visuell ähnlichsten Bilder zu ALASKA2 automatisch ausgewählt und im passenden Format organisiert.

Basierend auf diesen ausgewählten Bildern werden anschliessend drei **synthetische Stego-Varianten** erzeugt: `JMiPOD`, `JUNIWARD` und `UERD`. Diese Varianten entstehen durch gezielte, leichte **Modifikationen im DCT-Frequenzraum** (Y-Kanal, 8×8-Blöcke) und simulieren die statistischen Spuren echter Steganographiealgorithmen.

> **Hinweis:**  
> Die erzeugten Varianten enthalten keine eingebetteten Nachrichten, sondern imitieren lediglich statistisch ähnliche Spuren wie die echten Steganographie-Algorithmen.   
> Sie dienen ausschliesslich der Reproduzierbarkeit und der strukturellen Vergleichbarkeit zum echten Datensatz.

Eine ausführliche Beschreibung der Vorgehensweise findet sich im **Anhang A**.

In [ ]:
# Mit force_download=True wird die Datei immer heruntergeladen, auch wenn sie bereits existiert.
# Achtung: Der Ordner 'data/raw/PD12M/' wird geleert, bevor die neuen Daten heruntergeladen werden!
print(util.download_synthetic_PD12M(force_download=False))

In [ ]:
# Neue samplen aus dem PD12M Datensatz (ALASKA2 oder andere Referenzbilder sind notwendig)
print(util.build_pd12m_like_reference(cover_count=500, scan_limit=5_000))
# DCT-Stego-Varianten anlegen 
print(util.generate_stego_variants())

<table style="width:100%; background-color: white; padding: 10px; border-radius: 6px; box-shadow: 0 0 5px rgba(0,0,0,0.2);">
  <tr>
    <td>
      <h1 style="margin-bottom: 0; color: black; font-size: clamp(1.5rem, 2.5vw, 2.5rem);">
        Steganalyse mit Deep Learning auf dem ALASKA2 Datensatz
      </h1>
    </td>
    <td align="right">
      <img src="images/OST_Logo_DE_RGB@2000ppi.png" alt="OST Logo" width="180">
    </td>
  </tr>
</table>

**Autor:** Rino Albertin  
**Datum:** 27. April 2025

---
## Inhaltsverzeichnis

1. Einleitung  
2. Zielsetzung und Vorgehensweise  
3. Datenaufbereitung und EDA  
4. Modellarchitektur und Training  
5. Evaluation und Ergebnisse  
6. Fazit und Ausblick  
7. Referenzen und Eigenständigkeitserklärung

**Anhang**
<ol type="A">
  <li>Erzeugung des synthetischen Stego-Datensatzes</li>
  <li>JPEG-Kompression und DCT</li>
  <li>Steganographie-Algorithmen</li>
</ol>

---
## 1. Einleitung

Steganalyse beschäftigt sich mit dem Erkennen von in digitalen Medien versteckten Informationen. Im Kontext von Bildern bedeutet dies, Merkmale zu finden, die auf eine versteckte Nachricht hinweisen, ohne dass das Originalbild offensichtlich verändert erscheint. Mit dem wachsenden Einsatz von Deep Learning ergeben sich neue, leistungsfähige Methoden zur Identifikation solcher versteckten Strukturen.

Der [ALASKA2-Datensatz](https://www.kaggle.com/competitions/alaska2-image-steganalysis) ist ein Benchmark-Datensatz für moderne Bildsteganalysen und besteht aus 305.000 JPEG-Bildern, die zum Teil mit verschiedenen Steganographie-Verfahren manipuliert wurden. Ziel dieser Arbeit ist es, aktuelle Deep-Learning-Modelle zur Steganalyse praktisch anzuwenden, zu evaluieren und deren Leistungsfähigkeit aufzuzeigen.

---
## 2. Zielsetzung und Vorgehensweise

Ziel dieser Arbeit ist es, ein Deep-Learning-Modell zu entwickeln, das steganographisch veränderte Bilder im ALASKA2-Datensatz zuverlässig erkennt. Der Schwerpunkt liegt auf überwachten Lernverfahren (supervised learning), wobei geeignete neuronale Netzwerkarchitekturen verwendet werden.

Das Vorgehen gliedert sich in folgende Hauptschritte:
- **Datenaufbereitung und EDA:** Download, Vorbereitung und Analyse des ALASKA2-Datensatzes, einschliesslich Visualisierung und Untersuchung der Datenstruktur.
- **Modellarchitektur und Training:** Auswahl, Implementierung und Training geeigneter Deep-Learning-Modelle.
- **Evaluation und Ergebnisse:** Bewertung der Modelle anhand geeigneter Metriken und Visualisierung der Resultate.
- **Fazit und Ausblick:** Zusammenfassung der Erkenntnisse und mögliche Erweiterungen.

Aufgrund der Grösse des Datensatzes und limitierter lokaler Ressourcen erfolgten die ersten Analysen und die Entwicklung der Pipeline lokal auf 10% der Daten. Für das finale Training und die Auswertung auf dem vollständigen Datensatz wurde der GPU-Cluster der OST genutzt.

---
## 3. Datenaufbereitung und EDA

In diesem Kapitel werden die wesentlichen Eigenschaften des ALASKA2-Datensatzes, das Vorgehen bei der Datenaufbereitung sowie die ersten Analyseschritte (EDA) beschrieben. Ziel ist es, die Datenbasis für das Modelltraining strukturiert aufzubereiten und ein erstes Verständnis für die vorhandenen Muster und Besonderheiten zu gewinnen.

### 3.1 Datensatzstruktur

Der ALASKA2-Datensatz ist ein aktueller Benchmark für die Steganalyse von JPEG-Bildern. Er enthält insgesamt 300 000 gelabelte Trainingsbilder, die in vier gleich grosse Teilmengen aufgeteilt sind. Für jedes Motiv existieren vier Versionen – die unveränderte „Cover“-Version sowie drei mit verschiedenen Steganografie-Algorithmen manipulierte Varianten (`JMiPOD`, `JUNIWARD`, `UERD`). Die Länge der versteckten Nachricht (Payload) ist variabel und wird im Datensatz nicht explizit angegeben, jedoch so gewählt, dass die Schwierigkeit der Erkennung für alle Bilder vergleichbar bleibt. Alle Bilder liegen als JPEG mit identischer Auflösung in den Qualitätsstufen 75, 90 oder 95 und unter identischem Dateinamen in jeweils einem der vier Klassenordner vor.

Ein separater Testdatensatz mit 5 000 Bildern ohne Labels wird in dieser Arbeit nicht verwendet.

**Hinweis:** Die für diese Arbeit relevanten technische Details zu JPEG und DCT sowie zu den Steganographie-Algorithmen sind im Anhang B bzw. C erläutert.

### 3.2 Datenaufbereitung
Die Bilder werden anhand der Klassenordner gelabelt und zusammen mit den zugehörigen Dateipfaden erfasst. Anschliessend erfolgt eine zufällige und stratifizierte Aufteilung in Trainings-, Validierungs- und Testsets.

Ein wichtiger Aspekt bei dieser Aufteilung ist, dass alle vier Varianten eines Motivs (Cover, JMiPOD, JUNIWARD, UERD) immer gemeinsam in denselben Split (Train, Val oder Test) gelegt werden. Dies verhindert Information Leakage, da diese Varianten vom identischen Quellbild stammen und sich nur durch gezielte, schwache Modifikationen im DCT-Frequenzraum unterscheiden. Würden Cover- und Stego-Versionen desselben Motivs in unterschiedliche Splits fallen, könnte das Modell durch Wiedererkennung von Bildinhalten auf die Testdaten schliessen – was zu unrealistisch hoher Testgenauigkeit und schlechter Generalisierung führen könnte.

Eine saubere Trennung der Motive zwischen den Splits ist daher essenziell, um ein generalisierungsfähiges Modell zu trainieren.

In [ ]:
# sollen die syntetischen Daten genutzt werden?
FORCE_SYNTETIC_DATASET = False
# FORCE_SYNTETIC_DATASET = True

# Definiere die Pfade
alaska2_path = "data/raw/alaska2-image-steganalysis/Cover"
pd12m_path = "data/raw/PD12M/Cover"

# Funktion zum Prüfen, ob ALASKA2 vorhanden ist
def check_alaska2_exists(path: str) -> bool:
    return os.path.isdir(path) and any(f.lower().endswith(".jpg") for f in os.listdir(path))

# Wenn ALASKA2 vorhanden ist, wird er verwendet, ansonsten der synthetische PD12M-Datensatz
if check_alaska2_exists(alaska2_path) and not FORCE_SYNTETIC_DATASET:
    dataset_name = "ALASKA2"
    dataset_display_name = "ALASKA2"
    print("✅ ALASKA2-Datensatz gefunden.")
    cover_path = alaska2_path
    # Prozentualer Anteil der Bilder
    SUBSAMPLE_PERCENT = 0.10  # 10% lokal
else:
    dataset_name = "PD12M"
    dataset_display_name = "synthetischer PD12M-Datensatz"
    print("❌ ALASKA2-Datensatz nicht gefunden. Verwende stattdessen den synthetischen PD12M-Datensatz.")
    cover_path = pd12m_path
    SUBSAMPLE_PERCENT = 0.1  # 100%

In [ ]:
# Klassen und Labels definieren
CLASS_LABELS = {
    'Cover': 0,
    'JMiPOD': 1,
    'JUNIWARD': 2,
    'UERD': 3
}

# 1. Datensatz laden (inkl. Metadaten, Pfade, label_name)
dataset_df = util.prepare_dataset(Path(cover_path).parent, CLASS_LABELS, subsample_percent=SUBSAMPLE_PERCENT, seed=42)

# 2. Kopie für Modelltraining erstellen
dataset_numeric = dataset_df.copy()
dataset_numeric["label"] = dataset_numeric["label_name"].map(CLASS_LABELS)

# 3. Nur für EDA: label_name in sortierte, geordnete Categorical-Spalte umwandeln
label_order = ["Cover", "JMiPOD", "JUNIWARD", "UERD"]
dataset_df["label_name"] = pd.Categorical(dataset_df["label_name"], categories=label_order, ordered=True)

# 4. Split für Training
train_df, val_df, test_df = util.split_dataset_by_filename(dataset_numeric, train_size=0.8, val_size=0.1, test_size=0.1)

### 3.3 Explorative Datenanalyse (EDA)

Im Rahmen der EDA werden die Verteilungen der Klassen sowie der JPEG-Qualitätsstufen (75, 90, 95) analysiert, um ein besseres Verständnis über die Zusammensetzung des Datensatzes zu gewinnen. Zusätzlich werden exemplarische Bilder jeder Klasse visualisiert, inklusive direkter Vergleiche zwischen Cover- und Stego-Varianten desselben Motivs.

Ein zentrales Ziel ist die Identifikation potenzieller Stego-Artefakte – also subtiler statistischer oder visueller Unterschiede, die durch die Modifikation der DCT-Koeffizienten entstehen. Hierzu werden sowohl Bildstatistiken im Pixel- und Frequenzraum (DCT) untersucht als auch bildbasierte Embeddings zur explorativen Visualisierung (z. B. mittels t-SNE) verwendet.

Diese Analysen liefern erste Hinweise darauf, ob und wie stark sich die verschiedenen Stego-Varianten vom Original unterscheiden – sowohl für das menschliche Auge als auch aus statistischer Sicht. Die Erkenntnisse daraus bilden eine wichtige Grundlage für das anschliessende Modelltraining und die Wahl geeigneter Architekturmerkmale.

In [ ]:
# Caching-Konfiguration
USE_CACHE_SECTIONS = {
    "overview": False,
    "examples": False,
    "stats": False,
    "dct": False,
    "embedding": False,
    "diff": False,
    "split": False,
}

toggle = util.make_toggle_shortcut(dataset_df, dataset_name)

# Übersicht
overview_plots = [
    toggle("1-1. Struktur & Statistik", eda.eda_overview.show_dataset_overview),
    toggle("1-2. Klassenverteilung", eda.eda_overview.plot_class_distribution),
    toggle("1-3. JPEG-Qualitätsverteilung", eda.eda_overview.plot_jpeg_quality_distribution),
]

# Beispiele
example_plots = [
    toggle("2-1. Bildraster pro Klasse", eda.eda_examples.plot_image_grid),
    toggle("2-2. Vergleich Cover vs. Stego", eda.eda_examples.plot_cover_stego_comparison),
]

# Farbkanalstatistik
stat_plots = [
    toggle("3-1. Bild-Mittelwertverteilung", eda.eda_color_channel_statistics.plot_image_mean_distribution),
    toggle("3-2. KDE & Boxplot - YCbCr", eda.eda_color_channel_statistics.plot_kde_and_boxplot, color_space="YCbCr"),
    toggle("3-3. Korrelation YCbCr-Kanäle", eda.eda_color_channel_statistics.plot_channel_correlation),
    toggle("3-4. KDE & Boxplot - RGB", eda.eda_color_channel_statistics.plot_kde_and_boxplot, color_space="RGB"),
    toggle("3-5. Ausreisser (Z-Score)", eda.eda_color_channel_statistics.show_outliers_by_channel, z_thresh=3.0),
]

# DCT-Analyse
dct_plots = [
    toggle("4-1. DCT-Quantisierung (Cover + Δ)", eda.eda_dct.plot_dct_avg_and_delta),
    toggle("4-2. Anzahl DCT-Flips pro Bild", eda.eda_dct.plot_flip_counts),
    toggle("4-3. Verteilung und Saldo der DCT-Flips im Y-Kanal (AC, ±1)", eda.eda_dct.plot_flip_direction_overview),
    toggle("4-4. Flip-Verteilung nach DCT-Index", eda.eda_dct.plot_flip_position_heatmap),
    toggle("4-5. Flip-Masken Overlay", eda.eda_dct.plot_cover_stego_flipmask),
]

# Sektionen in Tabs gruppieren
sections = [
    util.make_dropdown_section(overview_plots, dataset_name, use_cache=USE_CACHE_SECTIONS["overview"]),
    util.make_dropdown_section(example_plots, dataset_name, use_cache=USE_CACHE_SECTIONS["examples"]),
    util.make_dropdown_section(stat_plots, dataset_name, use_cache=USE_CACHE_SECTIONS["stats"]),
    util.make_dropdown_section(dct_plots, dataset_name, use_cache=USE_CACHE_SECTIONS["dct"]),
]

tab_titles = [
    "1. Metadaten & Struktur",
    "2. Bildbeispiele",
    "3. Farbkanalstatistik",
    "4. DCT-Analyse",
]

# Hauptpanel anzeigen
eda_panel = util.make_lazy_panel_with_tabs(
    sections,
    tab_titles=tab_titles,
    open_btn_text=f"{dataset_display_name} EDA öffnen",
    close_btn_text="Schliessen",
)

display(eda_panel)

## 7. Referenzen und Eigenständigkeitserklärung

### 7.1 Referenzen
[1] https://www.kaggle.com/competitions/alaska2-image-steganalysis

https://www.kaggle.com/code/tanulsingh077/steganalysis-complete-understanding-and-model/notebook

https://arxiv.org/ftp/arxiv/papers/1704/1704.08378.pdf

https://utt.hal.science/hal-02542075/file/J_MiPOD_vPub.pdf

https://dde.binghamton.edu/vholub/pdf/Holub_PhD_Dissertation_2014.pdf

https://ijarcce.com/wp-content/uploads/2024/04/IJARCCE.2024.13478.pdf


@misc{alaska2-image-steganalysis,
    author = {Addison Howard and GIBOULOT Quentin and PatrickFrenchie and Rémi Cogranne and Will Cukierski},
    title = {ALASKA2 Image Steganalysis},
    year = {2020},
    howpublished = {\url{https://kaggle.com/competitions/alaska2-image-steganalysis}},
    note = {Kaggle}
}

### 7.2 Eigenständigkeitserklärung
Hiermit bestätige ich, dass ich die vorliegende Arbeit selbständig verfasst und keine anderen als die angegebenen Hilfsmittel benutzt habe.  
Die Stellen der Arbeit, die dem Wortlaut oder dem Sinn nach anderen Werken (dazu zählen auch Internetquellen) entnommen sind, wurden unter Angabe der Quelle kenntlich gemacht.

<table style="width:100%; background-color: white; padding: 10px; border-radius: 6px; box-shadow: 0 0 5px rgba(0,0,0,0.2); margin-top:20px;">
  <tr>
    <td align="left">
      <img src="images/Unterschrift.png" alt="Unterschrift" style="height:80px;">
    </td>
  </tr>
</table>

---
---

## Anhang

### A – Erzeugung des synthetischen Stego-Datensatzes

Zur Reproduzierbarkeit und öffentlichen Verfügbarkeit dieses Projekts wurde ein synthetischer Stego-Datensatz auf Basis des  **[PD12M (Public Domain 12 M)](https://source.plus/pd12m?size=n_100_n)-Datensatzes** erstellt. Da der ursprünglich verwendete **[ALASKA2-Datensatz](https://www.kaggle.com/competitions/alaska2-image-steganalysis)** nicht öffentlich weitergegeben werden darf, dient diese alternative Version der **Demonstration und strukturellen Vergleichbarkeit**.

Der PD12M-Datensatz steht unter **Public Domain / CC0** und enthält Millionen hochaufgelöster Fotos. Eine kuratierte Auswahl der *N* visuell ähnlichsten Bilder zu ALASKA2 ist öffentlich unter [Rinovative/pd12m_dct_based_synthetic_stegano](https://huggingface.co/datasets/Rinovative/pd12m_dct_based_synthetic_stegano) verfügbar und wird automatisch heruntergeladen.

#### Bilderauswahl

1. **Referenz-Embeddings**  
   - Auswahl von 300 Cover-Bildern aus ALASKA2  
   - CLIP (ViT-B/32) generiert 512-dimensionalen Embedding-Vektor pro Referenzbild  
2. **k-NN in Embedding-Raum**  
   - Streaming durch bis zu 10 000 Bilder aus PD12M  
   - CLIP-Embeddings für jedes Kandidatenbild berechnet  
   - L2-Normalisierung und Kosinus-Ähnlichkeit (Skalarprodukt) mit Referenz-Embeddings  
   - Min-Heap (Grösse = Anzahl gewünschter Cover, z.B. 500) führt Top-k Auswahl durch  
3. **Ergebnis**  
   - Die *k* Bilder mit höchsten Ähnlichkeitswerten werden übernommen

#### Vergleich zu echten Steganographie-Algorithmen

Die synthetisch erzeugten Varianten orientieren sich an den Charakteristika realer JPEG-Steganographiealgorithmen,  
nutzen jedoch bewusst vereinfachte Verfahren zur gezielten statistischen Modifikation ohne Nachrichteneinbettung.

| Aspekt | Echte Algorithmen | Synthetische Varianten |
|:-------|:------------------|:------------------------|
| **Arbeitsraum** | Frequenzraum (DCT der Y-Komponente) | Frequenzraum (DCT der Y-Komponente) |
| **Payload** | Nachrichteneinbettung (bits per DCT-Koeffizient) | Keine Nachricht, nur ±1-Modifikationen |
| **Verfahren** | Adaptive stochastische Modellierung | Frequenz- und texturbasierte ±1-Flips |
| **JMiPOD** | Wahrscheinlichkeitsmodell über Kostenfunktionen | ±1-Modifikationen im mittleren Frequenzbereich (2–6) |
| **JUNIWARD** | Strukturadaptive Hochfrequenz-Modulation | ±1-Flips in texturreichen Hochfrequenzbereichen |
| **UERD** | Gleichverteilte Zufallseinbettung über AC-Koeffizienten | Zufällige ±1-Modifikationen über alle AC-Koeffizienten |
| **Visuelle Auswirkungen** | Praktisch unsichtbar, schwer detektierbar | Visuell unauffällig, statistisch erkennbar |
| **Trainierbarkeit** | Sehr geringe Signalstärke, schwierig | Moderate Signalstärke, besser separierbar |

#### Modifikationslogik im Detail

Die synthetischen Varianten JMiPOD, JUNIWARD und UERD werden durch ±1-Flips auf den **quantisierten DCT-Koeffizienten des Y-Kanals** realisiert.  
Die Modifikationen erfolgen **direkt in der JPEG-Datei** (über `jpegio`), ohne Re-Komprimierung oder Inverse DCT. Nur nicht-null AC-Koeffizienten werden verändert. Die Varianten unterscheiden sich in der Auswahlmaske:

##### Vorbereitung: AC-Masken und Frequenzindizes

Es werden zunächst alle modifizierbaren Koeffizienten identifiziert. Dazu werden nullwerte und DC-Positionen (oben links in jedem 8×8 Block) ausgeschlossen. Anschliessend wird jedem Koeffizienten ein Frequenzindex (freq_sum) zugewiesen, der als Mass für seine spektrale Lage dient (niedrig = glatte Bildbereiche, hoch = feine Details).

```python
nz_mask = y_coef != 0
dc_mask = np.zeros_like(y_coef, dtype=bool)
dc_mask[0::8, 0::8] = True
ac_mask = nz_mask & ~dc_mask

u = np.tile(np.arange(h)[:, None], (1, w))
v = np.tile(np.arange(w)[None, :], (h, 1))
freq_sum = (u % 8) + (v % 8)  # Frequenzindex pro Koeffizient (0 = DC, 14 = HF)
```

##### Payload-Verteilung

Aus der Binärmaske der modifizierbaren Koeffizienten wird eine zufällige Teilmenge bestimmt, die der gewünschten Payload-Rate entspricht. Dies simuliert die Dichte an veränderten DCT-Koeffizienten, wie sie bei echten Einbettungen auftreten könnte.

```python
def _pick_payload_indices(mask, payload_rate, rng):
    candidates = np.column_stack(np.where(mask))
    n_bits = int(np.round(payload_rate * len(candidates)))
    if n_bits == 0:
        return np.empty((0, 2), dtype=int)
    sel = rng.choice(len(candidates), n_bits, replace=False)
    return candidates[sel]
```

##### JMiPOD-Variante

Es werden ausschliesslich DCT-Koeffizienten mittlerer Frequenz (Indexsumme zwischen 2 und 6) modifiziert. Dies entspricht der Strategie, unauffällige, aber informative Frequenzbereiche zu verändern. Die Modifikation besteht aus zufälligen ±1-Flips.

```python
jm_mask = (2 <= freq_sum) & (freq_sum <= 6) & ac_mask
idx = _pick_payload_indices(jm_mask, payload_rate=0.4, rng)
y_jm = y_coef.copy()
y_jm[idx[:, 0], idx[:, 1]] += rng.choice([-1, 1], size=len(idx))
```

##### JUNIWARD-Variante

Hier werden nur hochfrequente Koeffizienten in strukturierten Bildbereichen (texturreiche Blöcke mit hoher DCT-Energie) verändert. Dies ahmt die adaptive Zielauswahl von JUNIWARD nach, die besonders auf visuell komplexe Regionen fokussiert ist.

```python
hf_mask = (freq_sum > 6) & ac_mask
energy = np.abs(y_coef).reshape(h//8, 8, w//8, 8).sum(axis=(1, 3))  # Blockenergie
high_energy_blocks = energy > np.percentile(energy, 70)
block_mask = np.repeat(np.repeat(high_energy_blocks, 8, axis=0), 8, axis=1)
ju_mask = hf_mask & block_mask
idx = _pick_payload_indices(ju_mask, payload_rate=0.4, rng)
y_ju = y_coef.copy()
y_ju[idx[:, 0], idx[:, 1]] += rng.choice([-1, 1], size=len(idx))
```

##### UERD-Variante

Die Modifikationen erfolgen breit verteilt über alle nicht-null AC-Koeffizienten, ohne Frequenzgewichtung oder Blockselektion. Dies entspricht dem Konzept zufälliger, schwer vorhersagbarer Einbettungen mit hoher Streuung.

```python
ue_mask = ac_mask
idx = _pick_payload_indices(ue_mask, payload_rate=0.4, rng)
y_ue = y_coef.copy()
y_ue[idx[:, 0], idx[:, 1]] += rng.choice([-1, 1], size=len(idx))
```

#### Struktur des synthetischen Datensatzes

Die erzeugte Ordnerstruktur lautet:

```
/Cover/           → Ausgewählte PD12M-Cover (N Bilder, ähnlich zu ALASKA2)
/JMiPOD/          → Mid-Frequency ±1-Flips
/JUNIWARD/        → Texturmaskierte High-Frequency-Flips
/UERD/            → Zufällige ±1-Flips über alle AC-Koeffizienten
```

Die Dateinamen sind identisch (`00001.jpg`, `00002.jpg`, …), was eine direkte Zuordnung zwischen Cover und Stego-Varianten ermöglicht und die Struktur kompatibel zum ALASKA2-Format hält.

#### Wichtiger Hinweis

Die synthetischen Varianten enthalten **keine eingebetteten Nachrichten**, sondern simulieren lediglich die typischen Frequenzänderungen, wie sie bei echten Stego-Algorithmen auftreten könnten. Sie dienen ausschliesslich der **Reproduzierbarkeit**, **Trainierbarkeit** und **vergleichbaren Modellierung** von Steganalyse-Ansätzen.

#### Lizenz und Quellen

- **[Original PD12M-Datensatz:](https://source.plus/pd12m?size=n_100_n)** Public Domain / CC0  
- **[Synthetischer Stego-Datensatz:](https://huggingface.co/datasets/Rinovative/pd12m_dct_based_synthetic_stegano)** CC0 (verbleibende Public Domain)

---
### B. JPEG-Kompression und DCT

Die JPEG-Kompression ist das weltweit am häufigsten verwendete Verfahren zur verlustbehafteten Bildkompression. Ihr zentrales Element ist die **Diskrete Kosinustransformation (DCT)**, die das Bild von einer Pixel- in eine Frequenzdarstellung überführt.

#### Ablauf der JPEG-Kompression

1. **Farbraumtransformation:**  
   Das Originalbild wird zunächst vom RGB- in den YCbCr-Farbraum umgewandelt, wobei Y die Helligkeit und Cb/Cr die Farbinformationen repräsentieren. Im JPEG-Verfahren wird häufig ein **Subsampling der Farbinformationen (Cb/Cr)** vorgenommen, bei dem die Auflösung der Farbkanäle reduziert wird. Da das menschliche Auge für Helligkeit viel empfindlicher ist als für Farbdifferenzen, können die Farbinformationen stärker komprimiert werden, ohne dass das Bild an wahrgenommener Qualität verliert. Dieser Schritt führt zu einem Verlust von Farbdetails, die durch das Subsampling reduziert werden.

2. **Blockbildung:**  
   Das Bild wird in Blöcke der Grösse 8×8 Pixel unterteilt.

3. **Diskrete Kosinustransformation (DCT):**  
   Für jeden 8×8-Bildblock wird die DCT berechnet. Dadurch wird der Block aus dem Ortsraum (Pixelwerte) in den Frequenzraum überführt:  
   - Die DCT liefert **64 DCT-Koeffizienten**, von denen jeder einen bestimmten „Frequenzanteil“ im Block beschreibt.
   - Der **erste Koeffizient** (oben links in der Matrix, sog. **DC-Koeffizient**) steht für den durchschnittlichen Helligkeitswert des gesamten Blocks.
   - Die weiteren **AC-Koeffizienten** beschreiben immer feinere Details, Kanten und Texturen (Frequenzanteile in horizontaler, vertikaler und diagonaler Richtung).
   - Die Matrix ist so aufgebaut, dass die **niedrigen Frequenzen** oben links liegen (grobflächige Helligkeitsunterschiede), während die **hohen Frequenzen** (feine Details und Rauschen) nach unten rechts wandern.
   - Die meisten Bildinformationen sind in den niedrigen Frequenzen konzentriert, während viele hohe Frequenzanteile (unten rechts) sehr kleine Werte haben.

   Die DCT und ihre Inverse (IDCT) sind verlustfreie, mathematische Transformationen: Würden alle 64 Koeffizienten exakt gespeichert, könnte man den ursprünglichen Block perfekt rekonstruieren.

4. **Quantisierung:**  
   Die DCT-Koeffizienten werden mit einer Quantisierungstabelle abgerundet, was zu einem starken Informationsverlust vor allem bei hohen Frequenzen (feine Bilddetails) führt. Viele dieser Koeffizienten werden dabei zu Null, wodurch sich die Bilddaten stark komprimieren lassen. Für die Helligkeits- (Y) und die beiden Farbkanäle (Cb, Cr) werden dabei unterschiedliche Quantisierungstabellen verwendet: Die Tabelle für die Helligkeit ist feiner abgestuft, um möglichst viele Details zu erhalten, während bei den Farbinformationen eine gröbere Quantisierung zulässig ist, da das menschliche Auge Farbverluste weniger stark wahrnimmt. Die Quantisierung ist der zentrale Schritt, in dem beim JPEG-Verfahren die Kompression und der damit verbundene Qualitätsverlust stattfinden.

5. **Kodierung:**  
   Die quantisierten Koeffizienten werden abschliessend noch weiter komprimiert und gespeichert, um die Dateigrösse zu minimieren. Dieser Schritt erfolgt verlustfrei und beeinflusst die Bildinformation selbst nicht mehr.

#### Bedeutung der DCT für Steganalyse

Viele Steganographie-Algorithmen für JPEG-Bilder, wie sie auch im ALASKA2-Datensatz vorkommen, nutzen gezielt bestimmte DCT-Koeffizienten, um darin Informationen zu verstecken. Dabei werden meist nicht alle, sondern nur die weniger auffälligen (z. B. mittlere Frequenzen) modifiziert, um das Bild für das menschliche Auge möglichst unverändert erscheinen zu lassen. Die Einbettung von Stego-Informationen erfolgt bevorzugt im **Y-Kanal** (Helligkeit), da dieser eine höhere Auflösung und geringere Quantisierung aufweist. Die Farbkanäle (Cb, Cr) sind aufgrund ihrer stärkeren Quantisierung und Subsampling weniger geeignet, werden aber in einigen Fällen ebenfalls genutzt.

Veränderungen im DCT-Bereich sind für Deep-Learning-Modelle, die nur auf den rekonvertierten RGB-Bildern trainiert werden, oft schwer zu erkennen, da die Stego-Informationen im Frequenzraum verborgen sind.

**Zusammenfassend:**  
Die Kenntnis der JPEG-Kompression und insbesondere der DCT ist für die Steganalyse essenziell, da die Stego-Algorithmen ihre Informationen fast ausschliesslich in den DCT-Koeffizienten einbetten, insbesondere im Y-Kanal.

https://www.youtube.com/watch?v=n_uNPbdenRs&ab_channel=Computerphile

https://www.youtube.com/watch?v=Q2aEzeMDHMA&ab_channel=Computerphile

https://www.kaggle.com/c/alaska2-image-steganalysis/discussion/147494

#### Visualisierung der DCT-Frequenzbasis

Die folgende Abbildung zeigt die 64 DCT-Basisfunktionen für einen 8×8-Block. Jede Zelle stellt eine Frequenzkomponente dar, die das Muster beschreibt, das dieser Koeffizient im Bild erzeugt:

![DCT-Basisfunktionen](images/DCTjpeg.png)

- Oben links (heller Bereich) befinden sich die **niedrigen Frequenzen**, die grobe Helligkeitsunterschiede darstellen.
- Unten rechts (fein gemustert) befinden sich die **hohen Frequenzen**, die feine Details und Rauschen beschreiben.

Eine Animation verdeutlicht, wie ein Bildblock (der Buchstabe A) durch Addition einzelner DCT-Basisfunktionen aufgebaut werden kann:

![DCT-Animation](images/DCT-animation.gif)

Diese Darstellungen machen deutlich, warum Steganographie-Algorithmen bevorzugt mittlere bis hohe Frequenzen nutzen: Veränderungen in diesen Bereichen sind visuell weniger auffällig.

https://de.wikipedia.org/wiki/Diskrete_Kosinustransformation

---
### C. Steganographie-Algorithmen

Die Steganographie-Algorithmen im JPEG-Bereich arbeiten in der Regel im **DCT-Raum**, indem sie gezielt die DCT-Koeffizienten manipulieren. Diese Algorithmen nutzen den Frequenzbereich, um geheime Informationen zu verstecken, da Veränderungen in den mittleren und höheren Frequenzen für das menschliche Auge oft unsichtbar sind.
- **JMiPOD (JPEG Message in Pixel of DCT)** nutzt bevorzugt die mittleren Frequenzen der DCT, um Daten einzubetten. Die eingebetteten Informationen werden so verändert, dass sie für den menschlichen Betrachter kaum erkennbar sind.
- **JUNIWARD (JPEG Universal Wavelet Relative Distortion)** verwendet ein adaptives Verfahren, bei dem die DCT-Koeffizienten in einer Weise modifiziert werden, dass die Stego-Nachricht robust gegen Störungen bleibt und die visuelle Qualität des Bildes möglichst erhalten bleibt.
- **UERD (Unified Embedding and Reversible Data)** verwendet eine Methode zur **reversiblen Steganographie**, bei der die eingebetteten Informationen später exakt wiederhergestellt werden können. Auch hier werden gezielt DCT-Koeffizienten im unauffälligeren Frequenzbereich des Bildes verändert.

**Zusammenfassend:**  
Alle drei Algorithmen (JMiPOD, JUNIWARD, UERD) nutzen die **DCT-Koeffizienten** für die Datenversteckung und modifizieren gezielt die **mittleren und höheren Frequenzen**, um sicherzustellen, dass die Änderungen für das menschliche Auge kaum sichtbar sind. Die Kenntnis dieser Algorithmen und der DCT ist für die Steganalyse von JPEG-Bildern entscheidend.